# atomman.lammps.atom_dump.dump(system, fname, prop_info=None, xf='%.13e')

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2017-04-19

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The atom_dump.dump function writes a system's information to a LAMMPS dump style file. The dump file format provides a concise way of storing a large number of atoms and atom properties. In converting between atomman and dump files, this function allows for full control over the property name conversions and unit control in the form of a JSON data model.

Arguments:

- __system__ = the [atomman.System](atomman.System.ipynb) to extract values from.

- __fname__ = name (and location) of the file to write to.

Keyword Arguments:

- __prop_info__ = the data model defining how to relate the LAMMPS atomic attributes to the atomman atomic properties. If not specified, will try reading [fname].json.  If that doesn't work, a defaut model will be created based on the system and saved to [fname].json.

- __xf__ = c-style format string to use for floating point numbers. Default is '%.13e'. 

The underlying code can be found in [atomman/lammps/atom_dump.py](https://github.com/usnistgov/atomman/blob/master/atomman/lammps/atom_dump.py).

- - -

## Demonstration

Library imports

In [1]:
#Standard libraries
from __future__ import print_function
import os

#http://www.numpy.org/
import numpy as np    

#https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp

Create small demonstration system (GaAs unit cell) with multiple per-atom properties

In [2]:
prop_dict = {'atype':  [1, 1, 1, 1, 2, 2, 2, 2],
             'pos':   [[0.00, 0.00, 0.00],
                       [0.50, 0.00, 0.50],
                       [0.50, 0.50, 0.00],
                       [0.00, 0.50, 0.50],
                       [0.25, 0.25, 0.25],
                       [0.75, 0.75, 0.25],
                       [0.25, 0.75, 0.75],
                       [0.75, 0.25, 0.75]]}

atoms = am.Atoms(natoms=8, prop=prop_dict)
box = am.Box(a=5.65, b=5.65, c=5.65)
system = am.System(box=box, atoms=atoms, scale=True)
print(system)

avect =  [ 5.650,  0.000,  0.000]
bvect =  [ 0.000,  5.650,  0.000]
cvect =  [ 0.000,  0.000,  5.650]
origin = [ 0.000,  0.000,  0.000]
natoms = 8
natypes = 2
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.825 |   0.000 |   2.825
      2 |       1 |   2.825 |   2.825 |   0.000
      3 |       1 |   0.000 |   2.825 |   2.825
      4 |       2 |   1.413 |   1.413 |   1.413
      5 |       2 |   4.238 |   4.238 |   1.413
      6 |       2 |   1.413 |   4.238 |   4.238
      7 |       2 |   4.238 |   1.413 |   4.238


In [3]:
scalar = [0.,1.,2.,3.,4.,5.,6.,7.]
vector = [[1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4],
          [1, 2, 3, 4]]
tensor = [[[12.4, 10.1], [10.1, 3.0]],
          [[22.0,  9.8], [ 9.8, 1.0]],
          [[11.0, 13.1], [13.1, 3.4]],
          [[41.0, 18.9], [18.9, 2.1]],
          [[16.7, 14.7], [14.7, 1.6]],
          [[19.2,  8.9], [ 8.9, 2.1]],
          [[12.0, 13.1], [13.1, 2.4]],
          [[13.2, 11.2], [11.2, 1.5]]]

system.atoms_prop(key='scalar', value=scalar)
system.atoms_prop(key='vector', value=vector)
system.atoms_prop(key='tensor', value=tensor)

### Creating dump file without prop_info

Save system info to dump file. Set xf='%.3f' to reduce float precision and make it easier to see.

In [4]:
lmp.atom_dump.dump(system, 'test.dump', xf='%.3f')

Show contents of test.dump

In [5]:
with open('test.dump') as f:
    print(f.read())

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
8
ITEM: BOX BOUNDS pp pp pp
0.000000 5.650000
0.000000 5.650000
0.000000 5.650000
ITEM: ATOMS id type x y z scalar vector[1] vector[2] vector[3] vector[4] tensor[1][1] tensor[1][2] tensor[2][1] tensor[2][2]
1 1 0.000 0.000 0.000 0.000 1 2 3 4 12.400 10.100 10.100 3.000
2 1 2.825 0.000 2.825 1.000 1 2 3 4 22.000 9.800 9.800 1.000
3 1 2.825 2.825 0.000 2.000 1 2 3 4 11.000 13.100 13.100 3.400
4 1 0.000 2.825 2.825 3.000 1 2 3 4 41.000 18.900 18.900 2.100
5 2 1.413 1.413 1.413 4.000 1 2 3 4 16.700 14.700 14.700 1.600
6 2 4.238 4.238 1.413 5.000 1 2 3 4 19.200 8.900 8.900 2.100
7 2 1.413 4.238 4.238 6.000 1 2 3 4 12.000 13.100 13.100 2.400
8 2 4.238 1.413 4.238 7.000 1 2 3 4 13.200 11.200 11.200 1.500



As you can see, the scalar values remain the same, while vector and tensor values are given corresponding indexes. Also, since the vector values were given as integers they are represented as integers.

### Default prop_info model

Even though a prop_info data model wasn't specified, one was created and saved to the file test.dump.json. Its contents define the property name conversions that were performed on the data.

Note that the default prop_info created by [atom_dump.load](atomman.lammps.atom_dump.load) will not be exactly the same.

In [6]:
with open('test.dump.json') as f:
    print(f.read())

{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": null
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "scalar": {}, 
            "vector": {
                "shape": 4
            }, 
            "tensor": {
                "shape": [
                    2, 
                    2
                ]
            }
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos", 
                    "index": 0
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "index": 1
                }
            }, 
            "z": {
 

Having the default test.dump.json file also ensures that if test.dump is read back in to atomman using [atom_dump.load](atomman.lammps.atom_dump.load) that all the terms are converted back into the original property names. 

In [7]:
system = lmp.atom_dump.load('test.dump', prop_info='test.dump.json')
print(system)

avect =  [ 5.650,  0.000,  0.000]
bvect =  [ 0.000,  5.650,  0.000]
cvect =  [ 0.000,  0.000,  5.650]
origin = [ 0.000,  0.000,  0.000]
natoms = 8
natypes = 2
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.825 |   0.000 |   2.825
      2 |       1 |   2.825 |   2.825 |   0.000
      3 |       1 |   0.000 |   2.825 |   2.825
      4 |       2 |   1.413 |   1.413 |   1.413
      5 |       2 |   4.238 |   4.238 |   1.413
      6 |       2 |   1.413 |   4.238 |   4.238
      7 |       2 |   4.238 |   1.413 |   4.238


In [8]:
print(system.atoms_prop(key='scalar'))

[0 1 2 3 4 5 6 7]


__BUG!__ Float values of 'scalar' not retained!

In [9]:
print(system.atoms_prop(key='vector'))

[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]


In [10]:
print(system.atoms_prop(key='tensor'))

[[[ 12.4  10.1]
  [ 10.1   3. ]]

 [[ 22.    9.8]
  [  9.8   1. ]]

 [[ 11.   13.1]
  [ 13.1   3.4]]

 [[ 41.   18.9]
  [ 18.9   2.1]]

 [[ 16.7  14.7]
  [ 14.7   1.6]]

 [[ 19.2   8.9]
  [  8.9   2.1]]

 [[ 12.   13.1]
  [ 13.1   2.4]]

 [[ 13.2  11.2]
  [ 11.2   1.5]]]


### Customizing prop_info

The prop_info data model specifies how atomman interprets and converts properties between LAMMPS dump files and System objects. It provides the ability to fully define the metadata associated with a LAMMPS dump file so that terms and values can be appropriately interpreted without knowledge of LAMMPS settings and user-defined properties. And you only have to create one prop_info model for a specific simulation/System!

A demonstration is given below. More information on the meaning of the different terms can be found in the [prop_info Notebook](../data_models/prop_info.ipynb).

Rather than starting from scratch, it is easiest to call atom_data.dump to generate the default model, then fill in missing values.  Here, I've adapted the model above by 

1. Giving nm units for the box vectors (LAMMPS-dump-atoms_prop-relate -> box_prop -> unit) 
2. Specifying that vector should be in floats (LAMMPS-dump-atoms_prop-relate -> atoms_prop -> vector -> dtype) 
3. Giving nm units for the pos x,y,z terms (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute -> x,y,z -> relation -> unit) 
4. Giving GPa units for the scalar term (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute -> scalar -> relation -> unit) 
5. Renaming scalar and vector conversion names (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute)
6. Deleting the tensor information from both atoms_prop and LAMMPS-attribute

In [11]:
prop_info = """{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": "nm"
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "scalar": {}, 
            "vector": {
                "dtype": "float",
                "shape": 4                
            }
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos", 
                    "index": 0,
                    "unit": "nm"
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "index": 1,
                    "unit": "nm"
                }
            }, 
            "z": {
                "relation": {
                    "prop": "pos", 
                    "index": 2,
                    "unit": "nm"
                }
            }, 
            "singular": {
                "relation": {
                    "prop": "scalar",
                    "unit": "GPa"
                }
            }, 
            "first": {
                "relation": {
                    "prop": "vector", 
                    "index": 0
                }
            }, 
            "second": {
                "relation": {
                    "prop": "vector", 
                    "index": 1
                }
            }, 
            "third": {
                "relation": {
                    "prop": "vector", 
                    "index": 2
                }
            }, 
            "fourth": {
                "relation": {
                    "prop": "vector", 
                    "index": 3
                }
            }
        }
    }
}"""

Now, creating a dump file will reflect these changes:

- LAMMPS property names are now different
- box and position values scaled from angstroms to nm
- scalar/singular scaled from default atomman units to GPa
- tensor values are gone
- vector values still represented as ints (bug?)

In [12]:
lmp.atom_dump.dump(system, 'test.dump', xf='%.5f', prop_info=prop_info)
with open('test.dump') as f:
    print(f.read())

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
8
ITEM: BOX BOUNDS pp pp pp
0.000000 0.565000
0.000000 0.565000
0.000000 0.565000
ITEM: ATOMS id type x y z singular first second third fourth
1 1 0.00000 0.00000 0.00000 0 1 2 3 4
2 1 0.28250 0.00000 0.28250 160 1 2 3 4
3 1 0.28250 0.28250 0.00000 320 1 2 3 4
4 1 0.00000 0.28250 0.28250 480 1 2 3 4
5 2 0.14130 0.14130 0.14130 640 1 2 3 4
6 2 0.42380 0.42380 0.14130 801 1 2 3 4
7 2 0.14130 0.42380 0.42380 961 1 2 3 4
8 2 0.42380 0.14130 0.42380 1121 1 2 3 4



__File Cleanup__

In [13]:
os.remove('test.dump')
os.remove('test.dump.json')

- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)